# Pseudocode
- Read in excel file
- For each row
    - Determine language by reading text field (https://stackoverflow.com/questions/43377265/determine-if-text-is-in-english)
    - Clean Text
        - Write modifications to transformation workbook
        - Cases currently failing:
            id=423: 6.000
            id=687: ST ELMER'S
            id=19,694: 1 1/2"
            id=502,453: not being translated, we could try using the region of the plant to improve the translation success variance
    - If language is not english, translate to english
        - Write translations to transformation workbook
    - Split on spaces to create tokens
    - For each token, write row into new workbook. Same data as before but `text` is replaced by `token` and addl column `language`

# NOTE
It's nice to use this Colab notebook for understanding, but because the code is split into chunks, the workbooks are created and if we don't run those code chunks every time, the workbooks could be stuck in memory and the data output could be old.  
Therefore, it's recommended to ***run all code chunks CTRL+F9 every time you want to run the program***

# Pip Installations
Installs libraries that aren't native to Python core

In [160]:
!pip install langdetect
!pip install googletrans
!pip install openpyxl

# Imports
Imports libraries (Python core and 3rd party (installed via pip)) into our runtime environment

In [161]:
import html
import os
from langdetect import detect
from googletrans import Translator
from openpyxl import load_workbook, Workbook
from openpyxl.utils import get_column_letter

# Global Variables




In [162]:
# Constants (don't change)
WB_SOURCE_FILENAME = "Sample_Data.xlsx"
SOURCE_TEXT_COL = 5
WB_DEST_FILENAME = "Tokenized_Data.xlsx"
WB_TRANS_FILENAME = "Text_Transformations.xlsx"
# Variables (do change)
# Keeps track of which row to write to
write_row_index = 1
trans_row_index = 1
# Google requires we instantiate an instance of its Translator class for use
translator = Translator()

Function: print_progress  
This is just for printing the progress. Not important

In [163]:
import sys
def print_progress(num_current, num_total):
    sys.stdout.write('\r')
    sys.stdout.write(f"Processing {num_current}/{num_total}")
    sys.stdout.flush()

# Function: write_transformation
For any transformations we make with the `text` field such as translations, removing special characters, etc., we log them to a separate excel file for integrity auditing.

In [164]:
def write_transformation(sheet_trans, row_pk, type_str, text_old, text_new, *args):
    """
    Write Transformation to Tranformation Workbook
    """
    global trans_row_index
    sheet_trans.cell(row=trans_row_index, column=1).value = row_pk
    sheet_trans.cell(row=trans_row_index, column=2).value = type_str
    sheet_trans.cell(row=trans_row_index, column=3).value = text_old
    sheet_trans.cell(row=trans_row_index, column=4).value = text_new
    for i, arg in enumerate(args):
        sheet_trans.cell(row=trans_row_index, column=5 + i).value = arg
    trans_row_index += 1

# Fuction: decode_html
A text transformation function which detects HTML encodings and decodes them to unicode  
Example: `B&uuml;rot&uuml;r` -> `Bürotür`

In [165]:
def decode_html(row_i, text, sheet_trans):
    """
    Transform HTML encodings into Unicode
    """
    while "&" in text and ";" in text:
        code = text[text.find("&") + 1 : text.find(";")]
        code_lower = code.lower()
        if code_lower in html.entities.html5.keys():
            text_old = text
            text = text.replace(f"&{code};", html.entities.html5[code_lower])
            write_transformation(sheet_trans, row_i, "HTML DECODING", text_old, text)
        else:
            break
    return text

# Function: remove_special_chars
A text transformation function which detects special characters and removes them.  
This is largely dependent on how we want the text analysis to perform. Maybe we want commas, etc. to appear in the data for analysis, but for now, the common characters are removed.
We could have used a library for this to simply remove everything that is not `a-z`, but by manually listing the common characters, maybe we'll find something that stands out (for example, `(R)` or `(TM)` would have been transformed to just `R` and `TM` if we used a library)

**Known problems**
- Using " for inches (ex: #19: CROCODILE ACKAGING TAN 2")

In [166]:
def remove_special_chars(row_i, text, sheet_trans):
    """
    Remove special characters
    """
    for substr in [
        "(R)",
        "(TM)",
        "®",
        "(",
        ")",
        ",",
        ".",
        ":",
        ";",
        "[",
        "]",
        '"',
        "'",
        "+",
        "-",
        "&",
    ]:
        if substr in text:
            text_old = text
            text = text.replace(substr, " ")
            write_transformation(sheet_trans, row_i, "SPECIAL CHAR", text_old, text)
    return text

# Function: translate
Takes the text as input  
Uses the Google langdetect to detect the language  
If langage is not english, translates to english and records this in the text transformations excel sheet.

In [167]:
def translate(row_i, text, sheet_trans):
    """
    Detect row language and translate to English if necessary
    """
    row_lang = detect(text)
    text_temp = text
    if row_lang != "en":
        text = translator.translate(text).text
        write_transformation(
            sheet_trans,
            row_i,
            "TRANSLATION",
            text_temp,
            text,
            row_lang,
        )
    return text

# Function: get_tokens
Takes the text as input and splits it based on spaces to produce a list of tokens.  
Discards tokens with more than one character because not much value was found from single character tokens.  
For example: `a quick brown fox` -> `['quick', 'brown', 'fox']`

In [168]:
def get_tokens(text):
    split_text = text.split(" ")
    # Keep tokens with len > 1 (there are a lot of empty strings or single character results)
    tokens_list = list(filter(lambda x: len(x) > 1, split_text))
    return tokens_list

# The "Main" Function
In a traditional python file, the following sections would be consolidated into a single function. Instead below, we walk through each code "chunk" and explain it in sections



### Opening / Creating the workbooks
*   Open the Sample Data workbook
  * This already exists, so we must simply open it and get the first sheet (`.active`)
*   Create the destination workbook ("Tokenized_Data.xlsx")
*   Create the text transformations workbook
For the workbooks we're creating, we try to remove them first in order to delete old copies so if we're running this program multiple times, we don't have data overlapping. The Python operating system (`os`) throws an error if the file does not exists, which is why we include `except ... pass` which means just move on if that error is thrown.

In [169]:
# Open source workbook
wb_source = load_workbook(filename=WB_SOURCE_FILENAME)
sheet_source = wb_source.active

# Create destination workbook
try:
    os.remove(WB_DEST_FILENAME)
except OSError:
    pass
wb_dest = Workbook()
sheet_dest = wb_dest.active

# Create transformation workbook
try:
    os.remove(WB_DEST_FILENAME)
except OSError:
    pass
wb_trans = Workbook()
sheet_trans = wb_trans.active

### Iterating through the sample data

In [170]:
# Iterate through rows
num_rows = sheet_source.max_row
# we need to start counting at row '2' because openpyxl starts counting at 1
#   and also includes a row for the column numbers (metadata)
for row_i in range(2, num_rows):
    print_progress(row_i, num_rows - 1)

    # Ignore this, saving it in case we need it later
    #
    # Get every column of the row and put it into a python list
    # We use the openpyxl reference notation such as "A0:X0"
    source_row_values = [
        cell.value
        for cell in sheet_source[
            f"A{row_i}":f"{get_column_letter(sheet_source.max_column)}{row_i}"
        ][0]
    ]

    text = sheet_source[f"{get_column_letter(SOURCE_TEXT_COL + 1)}{row_i}"].value
    # Decode HTML characters before translation to improve translation detection
    text = decode_html(row_i, text, sheet_trans)
    text = translate(row_i, text, sheet_trans)
    # Remove special chars after translation, in case translator added some
    text = remove_special_chars(row_i, text, sheet_trans)
    # Make everything lowercase to improve analysis
    text = text.lower()

    tokens = get_tokens(text)
    # Write tokenized rows to dest workbook
    for token in tokens:
        # Written row will be: [row index, token, row_lang]
        dest_row_values = [row_i, token, row_lang]
        # openpyxl doesn't have a writerow() function so here's a nice O(n^3) loop
        for val_i, val in enumerate(dest_row_values):
            sheet_dest.cell(row=write_row_index, column=val_i + 1).value = val

        write_row_index += 1



Processing 1000/1000

### Saving the workbooks
So far, all the cell manipulations we've done (putting data into excel cells) has been in memory and not written to a file. Here we save the files to the filesystem.


In [171]:
# Ignore this, saving it for later
# Since we're downloading from Google Colab, the browser caches the files by name and returns old versions,
#   so append the current time to the filename (could have used uuid but this is more human friendly)
# from datetime import datetime
# now_strftime_extension = '_' + datetime.strftime(datetime.now(), '%y-%m-%d-%H-%M') + '.xlsx'
# wb_trans.save(filename=WB_TRANS_FILENAME.replace('.xlsx', now_strftime_extension))
# wb_dest.save(filename=WB_DEST_FILENAME.replace('.xlsx', now_strftime_extension))

wb_trans.save(filename=WB_TRANS_FILENAME)
wb_dest.save(filename=WB_DEST_FILENAME)